In [4]:
# GEt libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [5]:
# Import data
dataset = pd.read_csv('LinkedIn Profile Data.csv')
print(dataset.head())

  c_id  avg_time_in_previous_position  avg_current_position_length  \
0    1                       2.000000                        457.0   
1    2                       1.500000                        212.0   
2    3                       1.333333                        243.0   
3    4                       1.250000                        123.0   
4    5                       1.200000                        244.0   

   avg_previous_position_length                    m_urn  m_urn_id  \
0                        1338.0  urn:li:member:149435456      3008   
1                         897.5  urn:li:member:149435456      3008   
2                         669.0  urn:li:member:149435456      3008   
3                         562.5  urn:li:member:149435456      3008   
4                         474.6  urn:li:member:149435456      3008   

   no_of_promotions  no_of_previous_positions  current_position_length  age  \
0                 1                         1                      457   37   


C:\Users\jhm\AppData\Local\Temp\ipykernel_17960\2998624788.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('LinkedIn Profile Data.csv')


In [7]:
# Extract the "ethnicity" column, the head_pitch, head_yaw, and head_roll columns
# and the "promoted" column

# Remove the last column from the dataset (we do not care about face quality)
dataset = dataset.iloc[:, :-1]

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder = LabelEncoder()






In [ ]:
# Split to independent and dependent variables

X = dataset.iloc[:, :-1].to_numpy()
Y = dataset.iloc[:, -1].to_numpy()

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)